In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.linalg import svd

In [4]:
from urllib.parse import urlparse, parse_qs

#if i want to include SMA/stock data for same company, need to scrape function not just symbol
def TimeSeriesDaily_df(stock_name):
    """
    Read a CSV file from a URL and return a DataFrame with symbol as the last column.
    """
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={stock_name}&apikey=98WO4EK4CGOBTPQR&datatype=csv'
    qs = parse_qs(urlparse(url).query)
    symbol = qs["symbol"][0]
    df = pd.read_csv(url)
    df['symbol'] = symbol
    return df
    

In [ ]:
#Example: AAPL stock past 100 days daily time series data
aapl_df = TimeSeriesDaily_df('AAPL')
aapl_df.head()

,timestamp,open,high,low,close,volume,symbol
0,2025-05-15,210.95,212.9600,209.5400,211.45,45029473,AAPL
1,2025-05-14,212.43,213.9400,210.5801,212.33,49325825,AAPL
2,2025-05-13,210.43,213.4000,209.0000,212.93,51909332,AAPL
3,2025-05-12,210.97,211.2679,206.7500,210.79,63775814,AAPL
4,2025-05-09,199.00,200.5399,197.5350,198.53,36453923,AAPL


In [16]:
#19 home construction companies used in paper:
portfolio = ['FDX', 'UNP', 'KSU', 'NSC', 'UPS', 'CHRW', 'UAL', 'DAL', 'LUV', 'CSX', 'DHI', 'LEN', 'PHM', 'TOL', 'NVR', 'HD', 'LOW', 'SHW', 'SPY']

In [7]:
#would need to choose which data I want to use (open, high, low, close volumn) and it's easy to make a function that makes a series of this data
#Example: Close price
aapl_close = aapl_df['close'].rename('AAPL close')
msft_close = msft_df['close'].rename('MSFT close')
date = aapl_df['timestamp']
data = [date, aapl_close, msft_close]

In [8]:
#take multiple data (dtype pd.Series) and make into a data matrix
def make_data_matrix(data: list[pd.Series]):
    """ 
    Make M1 and M2 data matrices from a list of Series. The first element of the list
    must be the dates (time stamp) and the rest are the data series.
    """
    M1_df = pd.DataFrame(data[1:])
    M1_df.index = [s.name for s in data[1:]]
    M1_df.columns = data[0]
    M = M1_df.to_numpy()
    M1 = M[:, :-1]
    M2 = M[:, 1:]
    return M1, M2


In [9]:
M1, M2 = make_data_matrix(data)
M1

array([[211.45, 212.33, 212.93, 210.79, 198.53, 197.49, 196.25, 198.51,
        198.89, 205.35, 213.32, 212.5 , 211.21, 210.14, 209.28, 208.37,
        204.6 , 199.74, 193.16, 196.98, 194.27, 202.14, 202.52, 198.15,
        190.42, 198.85, 172.42, 181.46, 188.38, 203.19, 223.89, 223.19,
        222.13, 217.9 , 223.85, 221.53, 223.75, 220.73, 218.27, 214.1 ,
        215.24, 212.69, 214.  , 213.49, 209.68, 216.98, 220.84, 227.48,
        239.07, 235.33, 235.74, 235.93, 238.03, 241.84, 237.3 , 240.36,
        247.04, 247.1 , 245.55, 245.83, 244.87, 244.47, 244.6 , 241.53,
        236.87, 232.62, 227.65, 227.63, 233.22, 232.47, 232.8 , 228.01,
        236.  , 237.59, 239.36, 238.26, 229.86, 222.78, 223.66, 223.83,
        222.64, 229.98, 228.26, 237.87, 233.28, 234.4 , 236.85, 242.7 ,
        242.21, 245.  , 243.36, 243.85, 250.42, 252.2 , 255.59, 259.02,
        258.2 , 255.27, 254.49],
       [453.13, 452.94, 449.14, 449.26, 438.73, 438.17, 433.35, 433.31,
        436.17, 435.28, 425.4 ,

In [10]:
M1_centered = M1 - np.mean(M1)
U, s, Vt = np.linalg.svd(M1_centered)
S = np.zeros((U.shape[1], Vt.shape[0]))
np.fill_diagonal(S, s)
s_inv = np.linalg.inv(S)
A_tild = U.T@M2@Vt.T@s_inv



LinAlgError: Last 2 dimensions of the array must be square